In [245]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from pathlib import Path
from tratamento import trata_acomph, a0_a1

In [246]:
def get_prod():
    loc = Path('../ex_csv/produtibilidades/prod.csv')
    prod = pd.read_csv(loc, index_col=0)
    for i,row in prod.head(154).iterrows():
        row['prod'] = row['prod'].replace(",",".")
        row['prod'] = float(row['prod'])
    return prod

In [247]:
def importa_arquivos():
    acomph = trata_acomph.get_csv()
    produtibilidade = get_prod()
    a0, a1 = a0_a1.get_csv()
    local = Path('../ex_csv')
    local_post = local / 'postos.csv'
    postos = pd.read_csv(local_post, index_col=0)
    return acomph, a0, a1, postos, produtibilidade

In [248]:
def get_datas():
    meses = []
    for i in range(1,31):
        meses = [((date.today()-timedelta(days=i)).month)] + meses
    return meses

In [249]:
def ena_tipo_0(ena, acomph, produtibilidades, postos):
    vazoes_tipo0 = acomph.join(postos.query('tipo==0'),
                               on = 'posto', how = 'inner')
    vazoes_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo0 = produtibilidades.join(postos.query('tipo==0'),
                                                  on = 'posto', how = 'inner')
    produtibilidade_tipo0.drop(['nome','bacia','ree','sub_mer','tipo'], axis=1, inplace=True)
    for i in range(30):
        ena_dia = vazoes_tipo0.iloc[:,i].multiply(produtibilidade_tipo0['prod'], fill_value=0)
        ena.iloc[:,i] += ena_dia
    return ena

In [250]:
def ena_tipo_1(ena, acomph, produtibilidades, postos, a0, a1):
    meses = get_datas()
    vazoes_tipo1 = acomph.join(postos.query('tipo==1'),
                               on = 'posto', how = 'inner')
    vazoes_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                      axis=1, inplace=True)
    produtibilidade_tipo1 = produtibilidades.join(postos.query('tipo==1'),
                                                 on='posto', how='inner')
    produtibilidade_tipo1.drop(['nome','bacia','ree','sub_mer','tipo'],
                              axis=1, inplace=True)
    vazoes_base = trata_vazoes_base(acomph, a1)
    for i in range(30):
        A1 = a1.iloc[:,meses[i]]
        A0 = a0.iloc[:,meses[i]]
        X = vazoes_base.iloc[:,i]
        Y = A0 + (A1 * X)
        ena.iloc[:,i] += Y
    return ena

In [251]:
def trata_vazoes_base(acomph, a1):
    vazoes_base = acomph.loc[a1.iloc[:,0], :]
    vazoes_base.reset_index(inplace=True)
    base = a1.reset_index()
    indice = base.loc[:,'posto']
    vazoes_base.insert(0, "ind", indice, allow_duplicates=True)
    vazoes_base.drop('posto', axis=1, inplace=True)
    vazoes_base.rename(columns={'ind':'posto'}, inplace=True)
    vazoes_base.set_index('posto', inplace=True)
    vazoes_base.dropna(inplace=True)
    return vazoes_base

In [252]:
def cria_ena():
    acomph, a0, a1, postos, produtibilidades = importa_arquivos()
    ind = acomph.head(155).index
    col = (acomph.T).head(30).index
    ena = pd.DataFrame(0,index = ind, columns = col)
    ena += ena_tipo_0(ena,acomph,produtibilidades,postos)
    ena += ena_tipo_1(ena,acomph,produtibilidades,postos,a0,a1)
    return ena

In [253]:
e = cria_ena()
e.to_csv('../ex_csv/ena.csv')
e

,2019-09-29,2019-09-30,2019-10-01,2019-10-02,2019-10-03,2019-10-04,2019-10-05,2019-10-06,2019-10-07,2019-10-08,...,2019-10-19,2019-10-20,2019-10-21,2019-10-22,2019-10-23,2019-10-24,2019-10-25,2019-10-26,2019-10-27,2019-10-28
posto,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
